## Generate Underdog Fantasy +EV picks

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from datetime import datetime, timedelta
import re

from fuzzywuzzy import process
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

pd.set_option('display.max_columns', 500)

### Scraping

In [ ]:
import sys

sys.path.append('../')
from scrapers.pinnacle_scraper import PinnaclePropsScraper
from scrapers.underdog_scraper import UnderdogScraper

#### Underdog Fantasy
Must specify `UNDERDOG_USER` and `UNDERDOG_PASS` in `scrapers/utils/secrets.py`

In [ ]:
# setting testing = False turns on `headless` mode for selenium
uf_scraper = UnderdogScraper(testing=True)
# this takes a sec to run
uf_scraper.login()

In [ ]:
uf_scraper.scrape_odds(sports=['nba', 'nhl'])

Found 795 upcoming nba events on Underdog Fantasy.
Found 142 upcoming nhl events on Underdog Fantasy.


#### Pinnacle
Doesn't require login

In [ ]:
pin_scraper = PinnaclePropsScraper()

In [ ]:
pin_scraper.scrape_odds(['basketball', 'hockey'])

Scraped player props for Houston Rockets @ New York Knicks
Scraped player props for Miami Heat @ Toronto Raptors
Scraped player props for Orlando Magic @ Atlanta Hawks
Scraped player props for Charlotte Hornets @ New Orleans Pelicans
Scraped player props for Dallas Mavericks @ Los Angeles Lakers
Scraped player props for Brooklyn Nets @ Portland Trail Blazers
No player props found for Furman @ VMI
No player props found for Colgate @ Army
No player props found for Georgia State @ Appalachian State
Found 231 upcoming basketball events on Pinnacle.
Scraped player props for Montreal Canadiens @ New Jersey Devils
Scraped player props for Detroit Red Wings @ Florida Panthers
Scraped player props for Montreal Canadiens (Shots On Goal) @ New Jersey Devils (Shots On Goal)
Scraped player props for Detroit Red Wings (Shots On Goal) @ Florida Panthers (Shots On Goal)
No player props found for Guelph Storm @ Kitchener Rangers
No player props found for Brantford Bulldogs @ Oshawa Generals
No player p

### Preprocessing

In [ ]:
underdog_df = pd.concat([uf_scraper.odds_data['nba'], uf_scraper.odds_data['nhl']]).copy()
pinnacle_df = pd.concat([pin_scraper.odds_data['basketball'], pin_scraper.odds_data['hockey']]).copy()

In [ ]:
underdog_df['event'].unique()

array(['Points', 'Pts + Rebs + Asts', 'Assists', 'Rebounds',
       'Points + Assists', 'Points + Rebounds', 'Fantasy Points',
       'FT Made', 'Double Doubles', 'Rebounds + Assists',
       '3-Pointers Made', 'Turnovers', 'Blocks + Steals', 'Blocks',
       'Steals', 'Triple Doubles', 'Goals Against', 'Saves', 'Goals',
       'Shots', 'Power Play Points', 'Blocked Shots'], dtype=object)

In [ ]:
pinnacle_df['event'].unique()

array(['Assists', 'DoubleDouble', 'Points', 'PointsReboundsAssist',
       'Rebounds', 'ThreePointFieldGoals', 'Goals', 'ShotsOnGoal',
       'Saves'], dtype=object)

#### Line Matching
There are inevitable discrepancies in event names among sportsbetting platforms. Here we perform manual event name mapping coupled with fuzzy string matching to join these sets of odds data.

In [ ]:
# TODO: Improve string matching logic to minimize need for manual mapping
# TODO: Place the following functions in separate processing/utils file

def reformat_pinnacle(underdog_df, pinnacle_df):
    """
    Reformat Pinnacle dataframe to match Underdog dataframe's structure. 
    This maps Pinnacle event names to Underdog event names and processes the 'game' and 'event' columns to find the closest match using fuzzy string matching.

    :param underdog_df (DataFrame): Underdog fantasy data.
    :param pinnacle_df (DataFrame): Pinnacle data.
    :return (DataFrame): Reformatted Pinnacle dataframe.
    """
    u_df = underdog_df.copy()
    p_df = pinnacle_df.copy()
    
    p_df['event'] = p_df['event'].map({
        # bball
        'PointsReboundsAssist': 'Pts + Rebs + Asts',
        'ThreePointFieldGoals': '3-Pointers Made',
        'DoubleDouble': 'Double Doubles',
        'TripleDouble': 'Tripe Doubles',
        # football
        'PassReceptions': 'Receptions',
        'ReceivingYards': 'Receiving Yards',
        'RushingYards': 'Rushing Yards',
        'KickingPoints': 'Kicking Points',
        'TouchdownPasses': 'Passing TDs',
        'PassingYards': 'Passing Yards',
        'PassAttempts': 'Passing Attempts',
        # Hockey
        'ShotsOnGoal': 'Shots',
        
        
    }).fillna(p_df['event'])
    
    
    for col in ['game', 'event']:
        p_df[col] = p_df[col].apply(lambda x: process.extractOne(query = x, 
                                                                 choices = u_df[col].unique())[0])
    return p_df

def process_diff_lines(value_df):
    """
    Interpolate probabilities for lines that are unequal.
    This function adjusts probabilities for lines where Pinnacle and Underdog have different values, ensuring the probability reflects the line difference.
    :param value_df (DataFrame): Contains value bets with odds from both Pinnacle and Underdog.
    :return (DataFrame): Contains nterpolated probabilities and a flag indicating interpolation.
    """
    df = value_df.copy()
    df['interpolated'] = False
    mispriced_over_df = df[(df['line_pinn'] < df['line_uf']) & (df['o_u'] == 'under')]
    for idx, row in mispriced_over_df.iterrows():
        altered_prob = float(row['prob']) / float(row['line_pinn']) * float(row['line_uf'])
        df.loc[idx, 'prob'] = altered_prob
        df.loc[idx, 'interpolated'] = True
        
    mispriced_under_df = df[(df['line_pinn'] > df['line_uf']) & (df['o_u'] == 'over')]
    for idx, row in mispriced_under_df.iterrows():
        altered_prob = float(row['prob']) / float(row['line_pinn']) * float(row['line_uf'])
        df.loc[idx, 'prob'] = altered_prob
        df.loc[idx, 'interpolated'] = True
        
    return df

def find_value(underdog_df, pinnacle_df):
    """
    Find value bets by comparing Underdog and Pinnacle odds.
    This function merges the Underdog and Pinnacle dataframes, reformats the Pinnacle dataframe, and identifies value bets by comparing probabilities.


    :param underdog_df (DataFrame): Contains Underdog fantasy data.
    :param pinnacle_df (DataFrame): Contains Pinnacle data.
    :return (DataFrame): Contains value bets with relevant columns.
    """
    u_df = underdog_df.copy()
    p_df = pinnacle_df.copy()
    
    p_df = reformat_pinnacle(u_df, p_df)
    p_df['game'] = p_df['game'].apply(lambda x: "ARI @ CGY" if x == "LA @ DAL" else x )
    value_df = u_df.merge(p_df, on=['player', 'game', 'event'], suffixes=['_uf', '_pinn'])
    
    # stack dfs
    rel_cols = ['player', 'game', 'time', 'event', 'type', 'line_uf', 'line_pinn', 'prob', 'o_u'] 
    
    df_over = value_df.copy()
    df_under = value_df.copy()

    df_over['o_u'] = 'over'
    df_over['prob'] = df_over['over_prob']

    df_under['o_u'] = 'under'
    df_under['prob'] = df_under['under_prob']

    value_df = pd.concat([df_over, df_under], ignore_index=True)[rel_cols].sort_values('prob', ascending=False)
    value_df = value_df[(value_df['type'] != 'scorcher')]
    print(value_df.shape)
    value_df = value_df.drop_duplicates()
    print(value_df.shape)
    # value_df = process_diff_lines(value_df)
    
    return value_df

value_df = find_value(underdog_df, pinnacle_df).reset_index(drop=True)
value_df.head()

(132, 9)
(132, 9)


,player,game,time,event,type,line_uf,line_pinn,prob,o_u
0,Duncan Robinson,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,2.0,2.5,0.642857,under
1,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Assists,both,5.0,4.5,0.633700,over
2,Tyler Herro,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,3.0,3.5,0.621212,under
3,Spencer Dinwiddie,BKN @ POR,- 9:00PM CST,Assists,both,5.0,5.5,0.615385,under
4,Dennis Smith Jr.,BKN @ POR,- 9:00PM CST,Assists,both,5.0,4.5,0.615385,over


In [ ]:
process_diff_lines(value_df)

,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
0,Duncan Robinson,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,2.0,2.5,0.642857,under,False
1,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Assists,both,5.0,4.5,0.633700,over,False
2,Tyler Herro,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,3.0,3.5,0.621212,under,False
3,Spencer Dinwiddie,BKN @ POR,- 9:00PM CST,Assists,both,5.0,5.5,0.615385,under,False
4,Dennis Smith Jr.,BKN @ POR,- 9:00PM CST,Assists,both,5.0,4.5,0.615385,over,False
...,...,...,...,...,...,...,...,...,...,...
127,Spencer Dinwiddie,BKN @ POR,- 9:00PM CST,Assists,both,5.0,5.5,0.377373,over,True
128,Dennis Smith Jr.,BKN @ POR,- 9:00PM CST,Assists,both,5.0,4.5,0.563730,under,True
129,Tyler Herro,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,3.0,3.5,0.330943,over,True
130,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Assists,both,5.0,4.5,0.539113,under,True


### Calculating +EV Bets
[Underdog Fantasy Payout multiplier structure](https://underdogfantasy.zendesk.com/hc/en-us/articles/10276018891803-Pick-em-Insurance-Payout-Structure)


In [ ]:
from itertools import combinations

In [ ]:
# UF payout multiplier
payout_multipliers = {
    2: {'standard': 3, 'insured': 0}, 
    3: {'standard': 6, 'insured': 0},
    # 4: {'standard': 10, 'insured': 6},
    # 5: {'standard': 20, 'insured': 10},
}

def calc_EV(combo_prob, bet_size, bet_type):
    """
    Calculate the expected value (EV) of a bet combination.

    :param combo_prob (float): The combined probability of the bet combination.
    :param bet_size (int): The size of the bet combination.
    :type bet_size (str): The type of bet ('standard' or 'insured').
    :return (float): The expected value (EV) of the bet combination.
    """
    return (combo_prob * payout_multipliers[bet_size][bet_type]) - (1 - combo_prob)

def gen_bets(df, min_prob = 0.57):
    """
    This filters bets based on a minimum probability, generates all possible 
    bet combinations, calculates their probabilities, and computes the expected value (EV) 
    for each combination.

    :param df (DataFrame): Contains bets and their probabilities.
    :param min_prob (float): The minimum probability threshold for a bet to be considered (default is 0.57).
    :return (DataFrame): Contains all bet combinations, their EV, number of picks, bet type, and total probability, sorted by EV in descending order.
    """
    value_df = df.copy()
    value_df = value_df.query(f'prob>{min_prob}')
    print(value_df.shape)
    # display(value_df)
    all_bets = []

    for bet_size in payout_multipliers.keys():
        for bet_type in ['standard', 'insured']:
            combos = combinations(value_df.index, bet_size)
            for combo in combos:
                combo_prob = 1
                for bet in combo:
                    combo_prob *= value_df.loc[bet, 'prob']  # Multiplying probabilities
                combo_ev = calc_EV(combo_prob, bet_size, bet_type)
                all_bets.append({
                    'combo': combo,
                    'EV': combo_ev,
                    'num_picks':bet_size,
                    'bet_type':bet_type,
                    'total_prob':combo_prob
                })
    
    return pd.DataFrame(all_bets).reset_index(drop=True).sort_values('EV', ascending=False)

# Let's consider bets with a minimum total probability of 0.55
recomended_bets = gen_bets(value_df.query("line_pinn==line_uf"),
                           min_prob=0.55)
recomended_bets

# each combo tuple contains the indices of the events to bet on

(32, 10)


,combo,EV,num_picks,bet_type,total_prob
993,"(10, 11, 15)",0.446566,3,standard,0.206652
992,"(10, 11, 14)",0.446566,3,standard,0.206652
994,"(10, 11, 18)",0.433634,3,standard,0.204805
995,"(10, 11, 19)",0.429251,3,standard,0.204179
996,"(10, 11, 20)",0.429251,3,standard,0.204179
...,...,...,...,...,...
10886,"(40, 43, 44)",-0.831588,3,insured,0.168412
10908,"(43, 44, 45)",-0.832197,3,insured,0.167803
10909,"(43, 44, 46)",-0.832197,3,insured,0.167803
10910,"(43, 45, 46)",-0.832197,3,insured,0.167803


In [ ]:
def generate_unique_recs(df):
    """
    This filters out overlapping bet combinations to provide unique 
    recommendations based on the given dataframe of bet combinations.


    :param df (DataFrame): Contains bet combinations and their details.
    :return (DataFrame): Contains unique bet combinations without overlaps.
    """
    combo_df = df.copy()
    combo_sets = combo_df['combo'].apply(set).tolist()
    unique_combos_indices = []

    for i, combo_set in enumerate(combo_sets):
        if all(not combo_set.intersection(unique_combos_indices[j]) for j in range(len(unique_combos_indices))):
            unique_combos_indices.append(combo_set)
    unique_combo_df = combo_df[combo_df['combo'].apply(set).isin(unique_combos_indices)]

    return unique_combo_df

In [ ]:
unique_combo_df = generate_unique_recs(recomended_bets)
unique_combo_df.head()

,combo,EV,num_picks,bet_type,total_prob
993,"(10, 11, 15)",0.446566,3,standard,0.206652
1920,"(14, 18, 19)",0.372696,3,standard,0.196099
3353,"(20, 21, 23)",0.331144,3,standard,0.190163
222,"(22, 25)",0.303271,2,standard,0.325818
266,"(24, 26)",0.290640,2,standard,0.322660


This contains the sorted, recommended bets

In [ ]:
value_df.head(20)

,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
0,Duncan Robinson,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,2.0,2.5,0.642857,under,False
1,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Assists,both,5.0,4.5,0.633700,over,False
2,Tyler Herro,MIA @ TOR,- 6:30PM CST,3-Pointers Made,both,3.0,3.5,0.621212,under,False
3,Spencer Dinwiddie,BKN @ POR,- 9:00PM CST,Assists,both,5.0,5.5,0.615385,under,False
4,Dennis Smith Jr.,BKN @ POR,- 9:00PM CST,Assists,both,5.0,4.5,0.615385,over,False
5,Wendell Carter Jr.,ORL @ ATL,- 6:30PM CST,Rebounds,both,5.0,5.5,0.610895,under,False
6,Bam Adebayo,MIA @ TOR,- 6:30PM CST,Rebounds,both,12.0,12.5,0.607843,under,False
7,Mikal Bridges,BKN @ POR,- 9:00PM CST,Pts + Rebs + Asts,both,34.0,34.5,0.606299,under,False
8,Goga Bitadze,ORL @ ATL,- 6:30PM CST,Rebounds,both,7.0,7.5,0.600000,under,False
9,Tyler Herro,MIA @ TOR,- 6:30PM CST,Assists,both,4.0,3.5,0.596774,over,False


In [ ]:
# Store recommended bets
recomended_bets.to_csv('recommended_bets.csv')
unique_combo_df.to_csv('recommended_bets_unique.csv')
value_df.to_csv('value_df.csv')
recomended_bets = pd.read_csv('recommended_bets.csv')
unique_combo_df = pd.read_csv('recommended_bets_unique.csv')
value_df = pd.read_csv('value_df.csv')

In [ ]:
# Helper to output bet combinations
def read_recs(unique_combo_df, value_df, n=5):
    combos = unique_combo_df.copy().head(n)
    for idx, row in combos.iterrows():
        combo = row['combo']
        pick = value_df.iloc[list(combo)]
        print(f'Total Probability: {row["total_prob"]}')
        print(f'EV: {row["EV"]}')
        display(pick)
        print('-~'*50)        
read_recs(unique_combo_df, value_df, n=10)

Total Probability: 0.20665227953537788
EV: 0.4465659567476451


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
10,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Pts + Rebs + Asts,both,33.5,33.5,0.595142,under,False
11,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Rebounds,both,5.5,5.5,0.593496,under,False
15,Paolo Banchero,ORL @ ATL,- 6:30PM CST,Assists,both,5.5,5.5,0.585062,over,False


-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
Total Probability: 0.19609947030443045
EV: 0.3726962921310132


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
14,Jimmy Butler,MIA @ TOR,- 6:30PM CST,Points,both,22.5,22.5,0.585062,under,False
18,Bam Adebayo,MIA @ TOR,- 6:30PM CST,Pts + Rebs + Asts,both,39.5,39.5,0.579832,under,False
19,Luka Doncic,DAL @ LAL,- 7:30PM CST,Pts + Rebs + Asts,both,50.5,50.5,0.578059,under,False


-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
Total Probability: 0.19016345894880776
EV: 0.3311442126416544


...

-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
Total Probability: 0.31624913812916566
EV: 0.26499655251666265


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
29,Goga Bitadze,ORL @ ATL,- 6:30PM CST,Pts + Rebs + Asts,both,17.5,17.5,0.563319,under,False
34,Tyler Herro,MIA @ TOR,- 6:30PM CST,Points,both,22.5,22.5,0.561404,over,False


-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~
Total Probability: 0.3130082089697064
EV: 0.2520328358788255


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
35,Dennis Smith Jr.,BKN @ POR,- 9:00PM CST,Pts + Rebs + Asts,both,17.5,17.5,0.559471,under,False
36,Tyler Herro,MIA @ TOR,- 6:30PM CST,Pts + Rebs + Asts,both,32.5,32.5,0.559471,under,False


-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~


### Auto-clicking bets in Underdog Fantasy (Experimental)

In [ ]:
# testing
browser = webdriver.Chrome()

# TODO: Glitchy -- make more robust
def choose_pick(browser, pick_row):
    """
    WORK IN PROGRESS
    
    Leverages selenium and a list of bet recommendations to manually generate a bet-slip on Underdog Fantasy.

    :param browser (webdriver.Chrome): The Selenium WebDriver used to automate browser actions.
    :param pick_row: _description_
    """
    search_bar = browser.find_element(By.CSS_SELECTOR, 'input[data-testid="player-search-input"]')
    search_bar.clear()
    search_bar.send_keys(pick_row['player'])

    btns = browser.find_elements(By.CLASS_NAME, 'styles__pickEmButton__OS_iW')

    for idx, line in enumerate(browser.find_elements(By.CLASS_NAME, 'styles__overUnderListCell__tbRod')):
        line_text = line.text.split("\n")
        event_name = pick_row['line_uf'] + " " + pick_row['event']
        o_u = pick_row['o_u']
        
        if (event_name == line_text[0]) and ('Higher' in line_text) and ('Lower' in line_text):
            if o_u == "under":
                btns[0].click()
            else:
                btns[0].click()
            return True
        elif ('Higher' in line_text) and ('Lower' in line_text):
            btns.pop(0)
            btns.pop(0)
    return False

In [ ]:
# Sportsbetting lines are always moving. This ensures that auto-picked bets are not stale
combos = unique_combo_df.copy().head(5)
for idx, row in combos.iterrows():
    combo = row['combo']
    pick = value_df.iloc[list(combo)]
    for idx, pick_row in pick.iterrows():
        found = choose_pick(browser, pick_row)
        if not found:
            print(f'Line updated for {pick_row["player"]} - {pick_row["line_uf"]}')
    next = input("Click Enter for next bet...")
    if next != "":
        print('Exited...')
        break
    print('~-'*50)

,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
2,Bam Adebayo,MIA @ TOR,- 6:30PM CST,Rebounds,both,12.5,12.5,0.606299,under,False
8,Malik Beasley,MIL @ CLE,- 6:30PM CST,Pts + Rebs + Asts,both,15.5,15.5,0.578059,under,False
10,Evan Rodrigues,FLA vs DET,- 6:00PM CST,Points,both,0.5,0.5,0.576271,under,False


Line updated for Bam Adebayo - 12.5
Line updated for Malik Beasley - 15.5
~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
9,Bam Adebayo,MIA @ TOR,- 6:30PM CST,Pts + Rebs + Asts,both,39.5,39.5,0.576271,under,False
11,Victor Wembanyama,SAS @ BOS,- 6:30PM CST,Double Doubles,both,0.5,0.5,0.574468,under,False
12,Jeremy Sochan,SAS @ BOS,- 6:30PM CST,Points,both,12.5,12.5,0.572650,under,False


Line updated for Victor Wembanyama - 0.5
Line updated for Jeremy Sochan - 12.5
~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-~-


,player,game,time,event,type,line_uf,line_pinn,prob,o_u,interpolated
13,Sam Bennett,FLA vs DET,- 6:00PM CST,Points,both,0.5,0.5,0.570815,over,False
14,Brook Lopez,MIL @ CLE,- 6:30PM CST,Rebounds,both,5.5,5.5,0.568966,under,False


Line updated for Brook Lopez - 5.5
Exited...
